In [1]:
# Imports
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
from tempfile import TemporaryDirectory
import seaborn as sns
import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(1991)
torch.manual_seed(1991)
torch.cuda.manual_seed(1991)

import random
random.seed(1991)

import numpy as np
np.random.seed(1991)

Running on device: CUDA


In [2]:

# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:03<00:00, 49343891.64it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:


import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, Subset
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset

In [9]:
sch = 'linear'
init_rate = 0.3
init_method = 'snip_little_grad'
lr = 0.001
epoch = 5
weight_decay = 5e-4

from torch.optim.lr_scheduler import _LRScheduler

class LinearAnnealingLR(_LRScheduler):
    def __init__(self, optimizer, num_annealing_steps, num_total_steps):
        self.num_annealing_steps = num_annealing_steps
        self.num_total_steps = num_total_steps

        super().__init__(optimizer)

    def get_lr(self):
        if self._step_count <= self.num_annealing_steps:
            return [base_lr * self._step_count / self.num_annealing_steps for base_lr in self.base_lrs]
        else:
            return [base_lr * (self.num_total_steps - self._step_count) / (self.num_total_steps - self.num_annealing_steps) for base_lr in self.base_lrs]


def set_layer(model, layer_name, layer):
    splited = layer_name.split('.')
    if len(splited) == 1:
        setattr(model, splited[0], layer)
    elif len(splited) == 3:
        setattr(getattr(model, splited[0])[int(splited[1])], splited[2], layer)
    elif len(splited) == 4:
        getattr(getattr(model, splited[0])[int(splited[1])], splited[2])[int(splited[3])] = layer

class Masker(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, mask):
        ctx.save_for_backward(mask)
        return x

    @staticmethod
    def backward(ctx, grad):
        mask, = ctx.saved_tensors
        return grad * mask, None

class MaskConv2d(nn.Conv2d):
    def __init__(self, mask, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device='cpu'):
        super(MaskConv2d, self).__init__(in_channels, out_channels, kernel_size, stride,
                                     padding, dilation, groups, bias, padding_mode, device=device)
        self.mask = mask

    def forward(self, input):
        masked_weight = Masker.apply(self.weight, self.mask)
        return super(MaskConv2d, self)._conv_forward(input, masked_weight, self.bias)

@torch.no_grad()
def replace_maskconv(model):
    print("Remove Maskconv")
    for name, m in list(model.named_modules()):
        if isinstance(m, MaskConv2d):
            conv = nn.Conv2d(m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            conv.weight.data = m.weight
            conv.bias = m.bias
            set_layer(model, name, conv)

def get_grads_for_snip(model, retain_loader, forget_loader):
    indices = torch.randperm(len(retain_loader.dataset), dtype=torch.int32, device='cpu')[:len(forget_loader.dataset)]
    retain_dataset = torch.utils.data.Subset(retain_loader.dataset, indices)
    retain_loader = DataLoader(retain_dataset, batch_size=64, shuffle=True)

    model.zero_grad()
    for sample in retain_loader:
        inputs = sample[0]
        targets = sample[1]
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        outputs = model(inputs)
        loss = F.cross_entropy(outputs, targets)
        loss.backward()

    for sample in forget_loader:
        inputs = sample[0]
        targets = sample[1]
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        outputs = model(inputs)
        loss = -F.cross_entropy(outputs, targets)
        loss.backward()

@torch.no_grad()
def re_init_model_snip_ver2_little_grad(model, px): # re init smallest gradients
    print("Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)")
    for name, m in list(model.named_modules()):
        if isinstance(m, nn.Conv2d):
            mask = torch.zeros_like(m.weight, device=DEVICE).bool()
            nparams_toprune = round(px*mask.nelement())

            out_c, in_c, ke, _ = mask.shape
            value = -m.weight.grad.abs()
            topk = torch.topk(value.view(-1), k=nparams_toprune)
            mask.view(-1)[topk.indices] = True
            grad_mask = mask.clone().float()
            grad_mask[grad_mask==0] += 0.1

            new_conv = MaskConv2d(grad_mask, m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            nn.init.kaiming_normal_(new_conv.weight, mode="fan_out", nonlinearity="relu")

            new_conv.weight.data[~mask] = m.weight[~mask]

            set_layer(model, name, new_conv)


def unlearning(
    net,
    retain_loader,
    forget_loader,
    val_loader):
    if init_method=='snip_little_grad':
        # replace_maskconv(net)
        get_grads_for_snip(net, retain_loader, forget_loader)
        re_init_model_snip_ver2_little_grad(net, init_rate)

    """Simple unlearning by finetuning."""
    epochs = epoch
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=weight_decay)

    if sch=='linear':
        scheduler = LinearAnnealingLR(optimizer, num_annealing_steps=(epochs+1)//2, num_total_steps=epochs+1)
    net.train()

    for ep in range(epochs):
        for sample in retain_loader:
            inputs = sample[0]
            targets = sample[1]
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

        scheduler.step()
    #remove_prune(net)
    net.eval()
    return net


In [4]:
from sklearn.linear_model import LogisticRegression

def train_mia_on_class(model, class_index, train_set, test_set, no_class=True, splits_num=5):
  model = model.to("cpu")

  if not no_class:
    # Select indencies where class is class_index
    train_class_set = np.where(np.array(train_set.targets) == class_index)[0]
    test_class_set = np.where(np.array(test_set.targets) == class_index)[0]
  else:
    train_class_set = np.arange(len(train_set))
    test_class_set = np.arange(len(test_set))

  max_len = min([len(train_class_set), len(test_class_set)])
  # Make equal sizes
  train_class_set = train_class_set[:max_len]
  test_class_set = test_class_set[:max_len]

  # Obtain subsets
  train_class_set = torch.utils.data.Subset(train_set, train_class_set)
  test_class_set = torch.utils.data.Subset(test_set, test_class_set)


  # Make them
  class_test_loader = torch.utils.data.DataLoader(
    test_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  class_train_loader = torch.utils.data.DataLoader(
    train_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  criterion = nn.CrossEntropyLoss(reduction="none")
  # Obtain train and test logits
  logits_train = []
  for i, (images, labels) in enumerate(class_train_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_train.append(np.concatenate((logits.detach().numpy()[0], losses)))

  logits_test = []
  for i, (images, labels) in enumerate(class_test_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_test.append((np.concatenate((logits.detach().numpy()[0], losses))))

  logits_train, logits_test = np.array(logits_train), np.array(logits_test)
  # Create dataset
  ys = [1] * max_len + [0] * max_len
  ys = np.array(ys)
  p = np.random.permutation(len(ys))
  logits = np.concatenate((logits_train, logits_test))

  logits = logits[p]
  ys = ys[p]

  # Fit logitstic regression
  clf = LogisticRegression(random_state=0, max_iter=1000)
  cv = model_selection.StratifiedShuffleSplit(
        n_splits=splits_num, random_state=0
    )

  clf = model_selection.cross_val_score(
        clf, logits, ys, cv=cv, scoring="accuracy"
    )

  # Output score
  # print(f"Mean MIA attack score for class {class_index} is: " + str(clf.mean()))
  return clf.mean()

In [5]:


# Check accuracy

def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(loader, 0):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = net(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct / total

In [6]:

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    criterion = nn.CrossEntropyLoss(reduction="none")
    all_losses = []

    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        logits = net(inputs)
        losses = criterion(logits, targets).numpy(force=True)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)

## Plain network without regularization
## 5 %

In [10]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Baseline"
model_params = "cifar10_resnet18_baseline.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT -------
Model name: Resnet18 Baseline
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 94.8
MIA score before on forget set: 0.91


Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)
Retain set accuracy FORGET model: 78.4%
Test set accuracy FORGET model: 68.5%
Forget set accuracy FORGET model: 78.4%

The MIA has an accuracy of 0.860 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 98.4
MIA score before on forget set: 0.94


Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)
Retain set accuracy FORGET model: 78.6%
Test set accuracy FORGET model: 67.9%
Forget set accuracy FORGET model: 88.0%

The MIA has an accuracy of 0.896 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 89.60000000000001
MIA score before on forget se

In [11]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)


[0.91, 0.94, 0.84, 0.8400000000000001, 0.96, 0.89, 0.89, 0.89, 0.9099999999999999, 0.9099999999999999]
[0.86, 0.8960000000000001, 0.8320000000000001, 0.8480000000000001, 0.9120000000000001, 0.828, 0.8640000000000001, 0.8960000000000001, 0.8799999999999999, 0.8320000000000001]
[94.8, 98.4, 89.60000000000001, 84.0, 94.0, 88.0, 94.39999999999999, 96.8, 96.0, 95.6]
[78.37403276052657, 78.61722439955783, 78.20966315620227, 79.13468108195009, 78.7828848202263, 78.55448808136029, 79.31588890228707, 79.1603022751025, 78.61894330673847, 78.66789934882226]
[78.4, 88.0, 64.0, 64.4, 76.0, 64.8, 84.0, 76.4, 89.2, 78.0]
[68.45, 67.88, 68.89, 69.45, 69.58, 68.97, 69.58, 68.97, 69.36, 68.71000000000001]


In [12]:

mean_mia_score_before_baseline = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_baseline = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_baseline = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_baseline = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_baseline = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_baseline = np.mean(np.array(Accuracy_test))

In [13]:


print(mean_mia_score_before_baseline)
print(mean_mia_score_after_baseline)

print(mean_accuracy_forget_before_baseline)

print(mean_accuracy_forget_after_baseline)
print(mean_accuracy_retain_baseline)
print(mean_accuracy_test_baseline)

0.898
0.8648000000000001
93.16
76.32000000000001
78.74360081327737
68.98400000000001


## Dropout

In [14]:
def modify_model(model_train_reg):
  from collections import OrderedDict

  layer1 = model_train_reg.layer1

  for indx in range(len(layer1)):
    modified_layers = OrderedDict()
    for name, feature in layer1[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer1[indx] = nn.Sequential(modified_layers)



  layer2 = model_train_reg.layer2

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer2[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer2[indx] = nn.Sequential(modified_layers)

  layer3 = model_train_reg.layer3

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer3[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)
    model_train_reg.layer3[indx] = nn.Sequential(modified_layers)

  layer4 = model_train_reg.layer4

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer4[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer4[indx] = nn.Sequential(modified_layers)

  return model_train_reg

In [15]:


print("----- 5 PERCENT DROPOUT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Dropout"
model_params = "cifar10_resnet18_l2_dropout_regularization.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft = model_forget_ft.to(DEVICE)
  model_forget_ft = modify_model(model_forget_ft)

  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT DROPOUT -------
Model name: Resnet18 Dropout
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 86.4
MIA score before on forget set: 0.88


Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)
Retain set accuracy FORGET model: 73.3%
Test set accuracy FORGET model: 70.8%
Forget set accuracy FORGET model: 76.0%

The MIA has an accuracy of 0.908 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 85.2
MIA score before on forget set: 0.95


Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)
Retain set accuracy FORGET model: 73.8%
Test set accuracy FORGET model: 71.0%
Forget set accuracy FORGET model: 80.8%

The MIA has an accuracy of 0.944 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 67.60000000000001
MIA score before on fo

In [16]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.88, 0.95, 0.85, 0.88, 0.91, 0.88, 0.89, 0.89, 0.88, 0.9]
[0.908, 0.944, 0.8880000000000001, 0.8879999999999999, 0.8720000000000001, 0.8240000000000001, 0.8480000000000001, 0.8800000000000001, 0.9, 0.9399999999999998]
[86.4, 85.2, 67.60000000000001, 57.199999999999996, 74.4, 69.19999999999999, 80.80000000000001, 80.80000000000001, 88.0, 89.60000000000001]
[73.33896056915471, 73.84933874663344, 73.21916706866055, 73.94083125653188, 73.29715606471711, 73.50576791671692, 73.45700102498141, 72.90230520328798, 72.8621098538899, 73.30586816720258]
[76.0, 80.80000000000001, 52.0, 51.6, 72.39999999999999, 71.2, 78.4, 71.2, 84.0, 86.4]
[70.76, 71.02000000000001, 70.6, 71.45, 70.7, 70.67999999999999, 70.28999999999999, 70.02000000000001, 70.54, 71.03]


In [17]:


mean_mia_score_before_dropout = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_dropout = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_dropout = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_dropout = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_dropout = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_dropout = np.mean(np.array(Accuracy_test))

In [18]:


print(mean_mia_score_before_dropout)
print(mean_mia_score_after_dropout)

print(mean_accuracy_forget_before_dropout)

print(mean_accuracy_forget_after_dropout)
print(mean_accuracy_retain_dropout)
print(mean_accuracy_test_dropout)


0.8910000000000002
0.8892000000000001
77.92000000000002
72.4
73.36785058717764
70.70899999999999


## Augmentation model

In [19]:

# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
         transforms.RandomRotation(30), # Randomly rotate some images by 20 degrees
         transforms.RandomHorizontalFlip(), # Randomly horizontal flip the images
         transforms.ColorJitter(brightness = 0.1, # Randomly adjust color jitter of the images
                                                            contrast = 0.1,
                                                            saturation = 0.1),
         transforms.RandomAdjustSharpness(sharpness_factor = 2,
                                                                      p = 0.1), # Randomly adjust sharpness
          transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
        transforms.ToTensor(),
    ]
)

to_tensor_transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=to_tensor_transform
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [20]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Augmentation"
model_params = "cifar10_resnet18_augmentation.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_forget_ft.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)


----- 5 PERCENT -------
Model name: Resnet18 Augmentation
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 79.60000000000001
MIA score before on forget set: 0.9


Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)
Retain set accuracy FORGET model: 68.5%
Test set accuracy FORGET model: 71.7%
Forget set accuracy FORGET model: 72.0%

The MIA has an accuracy of 0.848 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 85.2
MIA score before on forget set: 0.92


Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)
Retain set accuracy FORGET model: 68.9%
Test set accuracy FORGET model: 71.8%
Forget set accuracy FORGET model: 83.2%

The MIA has an accuracy of 0.924 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 65.60000000000001
MIA score bef

In [21]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.8999999999999999, 0.9199999999999999, 0.78, 0.8200000000000001, 0.86, 0.8899999999999999, 0.9199999999999999, 0.86, 0.95, 0.8300000000000001]
[0.8480000000000001, 0.924, 0.8480000000000001, 0.86, 0.8720000000000001, 0.868, 0.8880000000000001, 0.884, 0.908, 0.86]
[79.60000000000001, 85.2, 65.60000000000001, 59.599999999999994, 80.80000000000001, 66.8, 81.6, 81.6, 88.8, 79.2]
[68.80853800699441, 68.87687422116815, 68.36023072130556, 68.36086659431649, 68.90639760416457, 69.00335631167474, 68.78956570670633, 68.30807573260441, 68.25837051328429, 68.56924653817553]
[74.4, 80.80000000000001, 58.4, 44.4, 64.8, 54.0, 73.2, 71.2, 82.39999999999999, 70.0]
[71.67999999999999, 71.76, 71.86, 71.61, 72.53, 72.23, 71.57, 72.0, 71.22, 71.22]


In [22]:


mean_mia_score_before_aug = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_aug = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_aug = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_aug = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_aug = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_aug = np.mean(np.array(Accuracy_test))

In [23]:


print(mean_mia_score_before_aug)
print(mean_mia_score_after_aug)

print(mean_accuracy_forget_before_aug)

print(mean_accuracy_forget_after_aug)
print(mean_accuracy_retain_aug)
print(mean_accuracy_test_aug)


0.873
0.8760000000000001
76.88
67.36
68.62415219503944
71.768
